In [1]:
import os
import sys
import django

# Async güvensiz modunu etkinleştir
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

# Django projesinin path'ini ekle
sys.path.append('/home/ocans/DJANGO-AI-AGENT/src')

# Django settings'i tanımla
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'cfehome.settings')

# Django'yu başlat
django.setup()

In [2]:
from mypermit import permit_client

permit = permit_client

In [3]:
await permit.api.users.list()

PaginatedResultUserRead(data=[UserRead(key='2', id=UUID('265dac99-510d-4428-835d-706aba106f29'), organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69'), project_id=UUID('ee7c3d54-1ff0-4eae-b7e1-dcabba85dc74'), environment_id=UUID('52a4ec82-92b9-4db0-bdc7-c9ecadcf4172'), associated_tenants=[], roles=[], created_at=datetime.datetime(2026, 2, 8, 18, 42, 37, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2026, 2, 8, 18, 48, 54, tzinfo=datetime.timezone.utc), email=None, first_name='staff', last_name=None, attributes=None), UserRead(key='3', id=UUID('c93ac430-540e-4aa2-84fb-528ac352cf71'), organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69'), project_id=UUID('ee7c3d54-1ff0-4eae-b7e1-dcabba85dc74'), environment_id=UUID('52a4ec82-92b9-4db0-bdc7-c9ecadcf4172'), associated_tenants=[], roles=[], created_at=datetime.datetime(2026, 2, 8, 18, 42, 38, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2026, 2, 8, 18, 48, 55, tzinfo=datetime.timezone.utc), email=No

In [4]:
from django.contrib.auth import get_user_model
User=get_user_model()

for u in User.objects.all():
    print(u.id, u.username) 
    user_data={
        "key" : f"{u.id}",
        "first_name": f"{u.username}"
    }  
    response = await permit.api.users.sync(user_data)
    print(response)

1 cfe
key='1' id=UUID('ee5699b5-d981-4bc2-a9fb-6b3d2dfa36ac') organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69') project_id=UUID('ee7c3d54-1ff0-4eae-b7e1-dcabba85dc74') environment_id=UUID('52a4ec82-92b9-4db0-bdc7-c9ecadcf4172') associated_tenants=[] roles=[] created_at=datetime.datetime(2026, 2, 8, 18, 42, 5, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2026, 2, 8, 18, 48, 53, tzinfo=datetime.timezone.utc) email=None first_name='cfe' last_name=None attributes=None
2 staff
key='2' id=UUID('265dac99-510d-4428-835d-706aba106f29') organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69') project_id=UUID('ee7c3d54-1ff0-4eae-b7e1-dcabba85dc74') environment_id=UUID('52a4ec82-92b9-4db0-bdc7-c9ecadcf4172') associated_tenants=[] roles=[] created_at=datetime.datetime(2026, 2, 8, 18, 42, 37, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2026, 2, 8, 18, 48, 54, tzinfo=datetime.timezone.utc) email=None first_name='staff' last_name=None attributes=None
3 ocans

In [5]:
await permit.check("1", "read", "document")

False

In [6]:
resources = [
    {
        "name": "Document",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {},
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_disovery",
        "actions": {
            "search": {},
            "detail": {},
        }
    }
]

In [7]:
from permit import PermitApiError

async def get_or_create_resource(resource_data:dict):
    resource_key = resource_data.get('key')
    try:
        _resource = await permit.api.resources.get(resource_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.resources.create(resource_data)
        else:
            raise e
    return _resource

In [8]:
for resource_data in resources:
    response = await get_or_create_resource(resource_data)
    print(response)

key='document' id=UUID('c1e52c90-62e1-4692-ab39-242e7a93b8a4') organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69') project_id=UUID('ee7c3d54-1ff0-4eae-b7e1-dcabba85dc74') environment_id=UUID('52a4ec82-92b9-4db0-bdc7-c9ecadcf4172') created_at=datetime.datetime(2026, 2, 9, 8, 21, 43, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2026, 2, 9, 8, 21, 43, tzinfo=datetime.timezone.utc) name='Document' urn='prn:ocansagbas:default:document' description=None actions={'read': ActionBlockRead(name='read', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('f28600fc-ee64-487b-93c3-18ea5b6758e3'), key='read'), 'create': ActionBlockRead(name='create', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('d1ba2298-e644-4320-add4-65889ddc7f50'), key='create'), 'update': ActionBlockRead(name='update', description=None, attributes=None, v1compat_path=None, v1compat_name=None, id=UUID('a049fd5b-6197-4bc7-996e-8775e6a3525f'

In [9]:
roles = [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": [
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }, 
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": [
            "document:read",
            "movie_disovery:search",
            "movie_disovery:detail"
        ]
    }
]

In [10]:
from permit import PermitApiError

async def get_or_create_role(role_data:dict):
    _key = role_data.get('key')
    try:
        _resource = await permit.api.roles.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.roles.create(role_data)
        else:
            raise e
    return _resource

In [11]:
for role_data in roles:
    response = await get_or_create_role(role_data)
    print(response)

name='Manager' description=None permissions=['movie_disovery:search', 'document:create', 'movie_disovery:detail', 'document:delete', 'document:update', 'document:read'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='manager' id=UUID('b9119a95-4407-419a-b7d3-99ef90197d5d') organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69') project_id=UUID('ee7c3d54-1ff0-4eae-b7e1-dcabba85dc74') environment_id=UUID('52a4ec82-92b9-4db0-bdc7-c9ecadcf4172') created_at=datetime.datetime(2026, 2, 9, 8, 22, 53, tzinfo=datetime.timezone.utc) updated_at=datetime.datetime(2026, 2, 9, 8, 22, 53, tzinfo=datetime.timezone.utc)
name='Viewer' description=None permissions=['movie_disovery:search', 'document:read', 'movie_disovery:detail'] attributes={} extends=[] granted_to=None v1compat_settings=None v1compat_attributes=None key='viewer' id=UUID('1f055906-a1cd-49dc-8033-a46283d65e58') organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69') project_id=UUID('

In [12]:
user = User.objects.all().first()

In [13]:
user.id

1

In [15]:
response = await permit.api.users.assign_role({
    "user": f"{user.id}",
    "role": "manager",
    "tenant": "default"
})
response

RoleAssignmentRead(id=UUID('aa9a117c-3682-40ff-9c92-9a4534e9faa2'), user='1', role='manager', tenant='default', resource_instance=None, resource_instance_id=None, user_id=UUID('ee5699b5-d981-4bc2-a9fb-6b3d2dfa36ac'), role_id=UUID('b9119a95-4407-419a-b7d3-99ef90197d5d'), tenant_id=UUID('270d3fe3-52a5-4874-bed5-413aa68f8175'), organization_id=UUID('4a39fbf7-bd81-4c28-9f34-8656767e9c69'), project_id=UUID('ee7c3d54-1ff0-4eae-b7e1-dcabba85dc74'), environment_id=UUID('52a4ec82-92b9-4db0-bdc7-c9ecadcf4172'), created_at=datetime.datetime(2026, 2, 9, 8, 26, 6, tzinfo=datetime.timezone.utc))

In [16]:
await permit.check(f"{user.id}", "delete", "document")

True

In [18]:
from langgraph.checkpoint.memory import InMemorySaver 
from AI.supervisors import get_supervisor
import uuid 

checkpointer = InMemorySaver()
supervisor = get_supervisor(model="gemini-2.5-flash", checkpointer=checkpointer)

In [19]:
doc_id = 4
user_id = user.id
user_id

1

In [20]:
config = {"configurable": {"user_id": user_id, "thread_id": uuid.uuid4()}}

response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": "what are my recent documents?"}
        ]
    },
    config
)

In [21]:
response['messages'][-1].content

[{'type': 'text',
  'text': 'Here are your recent documents:\n* Mission Impossible Summary (id: 4)\n* (id: 3)\n* Test Document (id: 2)\n* Test Document (id: 1)',
  'extras': {'signature': 'Ct8CAb4+9vu+HniVNqnFU+vhAwhX5PcHwamJuWeaae8OoaJ5FC5OFuDu40dMdzpAuNvVKnPDGV8ZAVKoAoTtaKZrweX74roht77bco90KlkqUR7b/SHbY4MYFI1VHvR4XCaBnsKCp6Z1Mq+S6XucaylNC6c608HQsFoSO7TCP9RvePDj6wsi8Mo0ODhJRnHuZILeREDI9FkHl9YIn6Jp5aVZ8H8/yohdKQxKepUVMk3old7VnuT3/oCwYe9n8vk5KwE28zDgkCBl6cd+lssySQX1jWs1DdG/N5wvAJSsBF7kEn0Q6UQKvxr5xoNhbqwE1OXx7KZwgB/XvbR153dXWm8deT2p10I41xd7Ni+m1dqwIAi4zRi8nmrZ+lhS0/1x+oVfBvmRTxN/mjNIwAS4DxOZtMNi62w9BlLEMkfMfaeVNA/ItJolnJBRAU+pRP1EnqoZmYG6XPYBqtDECSQqSUaZ'}}]

In [22]:
response = supervisor.invoke(
    {"messages": 
        [
            {"role": "user", "content": f"Delete any random document by id"}
        ]
    },
    config
)
response['messages'][-1].content

[{'type': 'text',
  'text': 'I need the document ID to delete a document. Please provide the ID of the document you want to delete.',
  'extras': {'signature': 'CuMGAb4+9vt/q6qggVemTXXdTTeHBUuKug3Hqh6JfooEcJW5hxKl+6rD4+rKaMau9NAoR5M8jDY/aUxk8E8DrwUL9voZwoiLlNk/to6Z2xzA1E6e4ih+nXmFj+ZRdvnIe1zZ8Jb4oNYF51Pp4hiNZnNAjLzTi1aF8faqwrJNvV4ug8lwOVwZx01rM1PsYnqDxR4AMPhxxnFohBAJ2wHshOH/0hpc5lExp2+xiwyeU1vUWelmvzx8TosbRDdSZqjsZdtidz/a+mx1/Q6XMXQWNt+t5mwO3TvynDN1oE+QW5xc1pVLi9TDBBv5cenWS4JknPgQ6Y0/u13E3sZoE+DTMJuNX8BzzPufwdlwcc1WWu/4SdTuWCV6Zm/bZrVknAlAlJP0KFIF9Ud386awT12yoZ5UpE8fKHvkkd6wj4K36tvmFFhFPFplcTv+dBaNFsJH3E3Rkdh4+lllKFVMz/qzv8F1Riy9OAwQNHUOHtmWKnfArnyTth31TiRhUZkdxcGTFMj0VfqEYYU/tvcA+tRdI5yHOyUgTCHWByPmVaGH8S+J0o8VLgtSZYuSegw/9QIWAMQjHwlDYQYOMZM/gDKF4tURXc7qKqeU2dFGlsGLdCWQRdl85ROm0NMH7F3fItYypmXBUcSJE2T4079NwO9OiQtR2fjKQoY9O7o+Dq5pUxdRf6PAPhJgoKlRe8p2ICqGM3UNResVmFyOMgl6u3UNK15qZ3N9L66jz2X6C6XoVU9qmCQVy+p7jE5HU2bpWgM0Lr2acVWFpJmas5t8fz4RZKoweOl3wdCCCVlq6PAUk9mMPSljVVLcZeeroqQg1FKeoRp66L/